# Caderno de testes e análises

 - Problema com minimização nas datas de vencimento dos bonds (2013, 2015 e 2017)
  - tentei atualizar o VNA para resolver esse problema
   - O problema resolveu algumas minimizações perto da data de vcto, mas ainda tem problema para o último dia antes do vcto
   
 - Quando um titulo vence, o vetor da fwd Original desloca uma coluna, preciso pensar em como corrigir esse deslocamento
     - Usei um dic(zip(vcto, txs))
     
 - O cálculo das taxas físicas estava apresentando uma inconsitência. Já corrigi
 - As taxas fixas calculadas são para ZCP, ou seja, é a taxa que desconta um vcto fixo, com base em hoje.

## Criação de uma função de interpolação de uma curva de juros a partir dos preços dos títulos

In [4]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
from scipy.optimize import *
import matplotlib.pyplot as plt


# Definindo as funções que serão utilizadas no trabalho


In [5]:

######################################################################################################
##################################### funcao para alocação dos fluxos numa matriz ####################
##########################################################1###########################################

def CalculaMatriz(dtToday, maxVcto, bonds, MONTHS_PGTO_COUPON, INPUT_FILE):
    # Para cálculo da matriz de vencimento das LTNs
    if ('LTN' in INPUT_FILE):
        PzPgtoCupom = []
        mtzCoupon = []
        for year in range(dtToday.year, maxVcto.year + 1, 1):
            for month in MONTHS_PGTO_COUPON:
                if (datetime(year, month, 1) > dtToday and datetime(year, month, 1) <= maxVcto):
                    PzPgtoCupom.append((datetime(year, month, 1) - dtToday).days)
                    vetorCoupon = []
                    for bond in bonds:
                        bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
                                         + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
                                         for i in range(0, bond['freqCoupon'])]
                        if (month in bondPmtMonths) and datetime(year, month, 1) == (datetime.strptime(bond['dtVcto'], '%Y-%m-%d')):
                            vetorCoupon.append(1)
                        else:
                            vetorCoupon.append(0)
                    mtzCoupon.append(vetorCoupon)
    
    elif ('Principal' in INPUT_FILE):        
    #NTNB Principal, ou seja, sem pagamento de cupom
        PzPgtoCupom = []
        mtzCoupon = []
        for year in range(dtToday.year, maxVcto.year + 1, 1):
            for month in MONTHS_PGTO_COUPON:
                if (datetime(year, month, 15) > dtToday and datetime(year, month, 15) <= maxVcto):
                    PzPgtoCupom.append((datetime(year, month, 15) - dtToday).days)
                    vetorCoupon = []
                    for bond in bonds:
                        bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
                                         + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
                                         for i in range(0, bond['freqCoupon'])]
                        if (month in bondPmtMonths) and datetime(year, month, 15) <= (datetime.strptime(bond['dtVcto'], '%Y-%m-%d')):
                            vetorCoupon.append(1)
                        else:
                            vetorCoupon.append(0)
                    mtzCoupon.append(vetorCoupon)
    
    else:
        # NTNB com pagamento de cupom
        PzPgtoCupom = []
        mtzCoupon = []
        for year in range(dtToday.year, maxVcto.year + 1, 1):
            for month in MONTHS_PGTO_COUPON:
                if (datetime(year, month, 15) > dtToday and datetime(year, month, 15) <= maxVcto):
                    PzPgtoCupom.append((datetime(year, month, 15) - dtToday).days)
                    vetorCoupon = []
                    for bond in bonds:
                        bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
                                         + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
                                         for i in range(0, bond['freqCoupon'])]
                        if (month in bondPmtMonths) and datetime(year, month, 15) <= (datetime.strptime(bond['dtVcto'], '%Y-%m-%d')):
                            vetorCoupon.append(1)
                        else:
                            vetorCoupon.append(0)
                    mtzCoupon.append(vetorCoupon)
    
    mtzCoupon = np.transpose(mtzCoupon)
    return mtzCoupon, PzPgtoCupom

######################################################################################################
##################################### funcao para alocação do principal na matriz de prazos ##########
######################################################################################################

def CalculaMatrizPrincipal(VNA,PzPgtoCupom, bonds, INPUT_FILE):
    # criando a matriz de principais
    mtzPrincipal = []
    if ('LTN' in INPUT_FILE):
        for k, bond in enumerate(bonds):
            mtzPrincipal.append(1000 * (np.array(PzPgtoCupom) == (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days))
    elif('Principal' in INPUT_FILE):
        for k, bond in enumerate(bonds):
            mtzPrincipal.append(VNA * (np.array(PzPgtoCupom) == (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days))
    else:
        for k, bond in enumerate(bonds):
            mtzPrincipal.append(VNA * (np.array(PzPgtoCupom) == (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days))
    return mtzPrincipal

######################################################################################################
##################################### funcao subtraindo o preço do bond (minimização) ################
######################################################################################################

def PrecoBondMatrizMin(xGuess):
    # Essa função recebe como input as taxas dos forwards em base a.a (360) e retorna a diferença 
    # entre o valor do bond calculado por DCF por essas taxas e o preço do bond no mercado
    xGuess = list(reversed(xGuess))
    DCMat = np.zeros(len(PzPgtoCupom))
    for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
    PzMat = np.mat(PzPgtoCupom)/360
    DFMat = np.power(DCMat,PzMat)
    F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) - np.mat(b).T
    F = np.asarray(F).reshape(-1)
#     print(F,b)
    return F

######################################################################################################
##################################### funcao sem subtrair o preço do bond ############################
######################################################################################################

def PrecoBondMatriz(xGuess):
    # Essa função recebe como input as taxas dos forwards em base a.a (360) e retorna o valor do bond calculado
    # por DCF por essas taxas
    xGuess = list(reversed(xGuess))
    DCMat = np.zeros(len(PzPgtoCupom))
    for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
    PzMat = np.mat(PzPgtoCupom)/360
#     print(PzPgtoCupom)
#     print(DCMat)
    DFMat = np.power(DCMat,PzMat)
    print(mtzVlCupom, DFMat)
    F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) # - np.mat(b).T  
    F = np.asarray(F).reshape(-1)
#     print(F)
    return F

######################################################################################################
##################################### funcao para calculo das taxas com prazos fixos #################
######################################################################################################

def CalculaFwdPzFixo(fwdsOriginais, vctosBonds, dtBase, vctosFixos):
    # Essa função está calculando a taxa de desconto ZCB para cada vcto fixo passado
    # Deveria adaptar para cálcular as fwds?
    # Calcula os prazo em DC de cada um dos vencimentos
    vctosDC = []
    for vctosBond in vctosBonds:
        vctosDC.append((vctosBond - dtToday).days)
#     print(vctosDC)
    # instancia a list com os fwds fixos
    fwd = [1] * len(vctosFixos)
    
    for pz in vctosFixos: # para cada prazo, verifica qual é maior ou menor que todos os vencimentos dos bonds
        for i in range(len(vctosBonds)):
            
            # preciso levar em contar que fwd vale entre os prazos, não da data base para o vcto

            if i == 0:
                if pz < vctosDC[i]:
                    fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) 
                                                                                     ** (pz/360))
#                     print(pz, vctosDC[i], fwd[listDtVctoFixo.index(pz)], fwdsOriginais[i], pz)
                    break
                else:
                    fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) 
                                                                                     ** (vctosDC[i]/360))
#                     print(pz, vctosDC[i], fwd[listDtVctoFixo.index(pz)], fwdsOriginais[i], vctosDC[i])
            else:
                if pz < vctosDC[i]:
                    fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) 
                                                                                     ** ((pz - vctosDC[i-1])/360))
#                     print(pz, vctosDC[i], fwd[listDtVctoFixo.index(pz)], fwdsOriginais[i], (pz - vctosDC[i-1]))
                    break
                else:
                    if i == len(vctosBonds) - 1:
                        fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) 
                                                                                     ** ((pz - vctosDC[i])/360))
#                         print(pz, vctosDC[i], fwd[listDtVctoFixo.index(pz)], fwdsOriginais[i], (pz - vctosDC[i]))
                    else:
                        fwd[listDtVctoFixo.index(pz)] = fwd[listDtVctoFixo.index(pz)] * ((1 + fwdsOriginais[i]) 
                                                                                     ** ((vctosDC[i] - vctosDC[i-1])/360))
#                         print(pz, vctosDC[i], fwd[listDtVctoFixo.index(pz)], fwdsOriginais[i], (vctosDC[i] - vctosDC[i-1]))
        
#         print(fwd[listDtVctoFixo.index(pz)], pz)
        fwd[listDtVctoFixo.index(pz)] = (fwd[listDtVctoFixo.index(pz)] ** (360/pz)) - 1
#         print(pz, vctosDC[i], fwd[listDtVctoFixo.index(pz)])
    return fwd

######################################################################################################
##################################### função para atualizar o VNA ####################################
######################################################################################################

def AtualizaVNA(DtHoje):
    # Função que recebe a data base e atualiza do VNA projetando a inflação aferida na data passada
    # Essa atualização deixa os valores dos preços calculados mais próximos do mercado quando perto
    # do vcto de um bond
    DtHoje = pd.to_datetime(DtHoje)
    dfVNA.index = pd.to_datetime(dfVNA.index.tolist())
    VNAt = dfVNA[dfVNA.index <= DtHoje].max()['VNA']
    VNAt_1 = dfVNA[dfVNA['VNA'] < VNAt].max()['VNA']
    DtVNAt = dfVNA.index[dfVNA['VNA'] == VNAt]
    DtVNAtD1 = dfVNA.index[dfVNA['VNA'] < VNAt][-1]
    DtProxVNA = dfVNA.index[dfVNA['VNA'] > VNAt][0]
    
    PzDCHoje_UltVNA = (DtHoje - DtVNAt).days
    PzDCProxVNA_UltVNA = (DtProxVNA - DtVNAt).days
#     print(DtProxVNA, VNAt/VNAt_1, PzDCHoje_UltVNA , PzDCProxVNA_UltVNA, ((VNAt/VNAt_1) ** ((PzDCHoje_UltVNA / (PzDCProxVNA_UltVNA)))))
    VNAt = VNAt * ((VNAt/VNAt_1) ** ((PzDCHoje_UltVNA / (PzDCProxVNA_UltVNA))))
    return VNAt.tolist()[0]

# Início do cálculo das taxas forwards Reais

In [12]:
# Definindo algumas variáveis
INPUT_FILES = ['NTNB_2013.csv','NTNB_2014.csv', 'NTNB_2015.csv', 'NTNB_2016.csv', 'NTNB_2017.csv', 'NTNB_2018.csv',  ]
INPUT_PATH = "D:\\Documents\\JupyterNB\\tese_fgv\\dados\\dados_tratados\\"
INPUT_FILE = INPUT_FILES[1]
VNA_FILE = "VNAHist.csv"
OUTPUT_FILE = "fwdReal_" + INPUT_FILE.split(".")[0].split("_")[1] + ".csv"
OUTPUT_FILE_REAL = "fwdRealOriginais_" + INPUT_FILE.split(".")[0].split("_")[1] + ".csv"
MONTHS_PGTO_COUPON = [2, 5, 8, 11]
listDtVctoFixo = [360, 5*360, 10*360, 20*360]

# Importando dados
dfPrecos = pd.read_csv(INPUT_PATH + INPUT_FILE, index_col='Dia')
dfVNA = pd.read_csv(INPUT_PATH + VNA_FILE, index_col = 'DataBase')

## Calculo para as NTNBs

In [13]:
vcBonds = list(dfPrecos.columns)
xGuess = [0.05] * len(vcBonds)
outputFwd  = []
outputFwdOriginais = []
# TESTA SE É O ARQUIVO É DE NTNB (com fluxo de caixa) OU LTN (sem fluxo de caixa)
if ('LTN' in INPUT_FILE):
    freqCupom = 0
else:
    freqCupom = 2

i_aux = 0 # utilizado para fazer pontos de parada no for
# Inicio do calculo das fwds para cada um dos dias
# for index, row in dfPrecos.loc['2013-05-14':'2013-05-14'].iterrows():
for index, row in dfPrecos.iterrows():
    bonds = []
    b =[]
    
    # Inicia a data base e VNA
    dtToday = datetime.strptime(index, "%Y-%m-%d")
    VNA = AtualizaVNA(index)
#     print(row)
    for vcBond in vcBonds:
        # Cria o dicionário com os preços e vencimentos para cada dia
        if datetime.strptime(vcBond, '%d/%m/%Y').strftime('%Y-%m-%d') >= index and not pd.isna((row[vcBond])): # verifica se tem algum bond vencido 
            bonds.append({'preco':row[vcBond], 'dtVcto':datetime.strptime(vcBond, '%d/%m/%Y').strftime('%Y-%m-%d'),
                           'freqCoupon': freqCupom})
            b.append(row[vcBond])    

    # Capturo algumas informações para o calculo das matrizes
    dtVctoBond = []
    for bond in bonds:
        dtVctoBond.append(datetime.strptime(bond['dtVcto'], '%Y-%m-%d'))

    maxVcto = max(dtVctoBond)
    dayVcto, monthVcto, yearVcto = maxVcto.day, maxVcto.month, maxVcto.year
    dayBase, monthBase, yearBase = dtToday.day, dtToday.month, dtToday.year

    # Calculo da matriz de cupons
    mtzCoupon, PzPgtoCupom = CalculaMatriz(dtToday, maxVcto, bonds, MONTHS_PGTO_COUPON, INPUT_FILE)
    
#     # Calculo da matriz de cupons
#     PzPgtoCupom = []
#     mtzCoupon = []
#     for year in range(dtToday.year, maxVcto.year + 1, 1):
#         for month in MONTHS_PGTO_COUPON:
#             if (datetime(year, month, 15) > dtToday and datetime(year, month, 15) <= maxVcto):
#                 PzPgtoCupom.append((datetime(year, month, 15) - dtToday).days)
#                 vetorCoupon = []
#                 for bond in bonds:
#                     bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
#                                      + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
#                                      for i in range(0, bond['freqCoupon'])]
#                     if (month in bondPmtMonths) and datetime(year, month, 15) <= (datetime.strptime(bond['dtVcto'], '%Y-%m-%d')):
#                         vetorCoupon.append(1)
#                     else:
#                         vetorCoupon.append(0)
#                 mtzCoupon.append(vetorCoupon)
#     mtzCoupon = np.transpose(mtzCoupon)
    
    # criando o vetor de cupons
    vlCupom = ((1 + 0.06)**(0.5) -1) * VNA
    arrayvlCupom = [vlCupom] * len(PzPgtoCupom)
#     print(vlCupom)

    # criando a matriz de principais
    mtzPrincipal = CalculaMatrizPrincipal(VNA,PzPgtoCupom, bonds, INPUT_FILE)
#     mtzPrincipal = []
#     for k, bond in enumerate(bonds):
#         mtzPrincipal.append(VNA * (np.array(PzPgtoCupom) == (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days))

    # Somando a matriz de cupons e de principais
    mtzVlCupom = (arrayvlCupom * mtzCoupon) + mtzPrincipal

    # Calculo das fwds para os vctos dos bonds (taxas a.a base 360)
#     xGuess = [0.015] * len(bonds)
#     print(len(xGuess), len(bonds))
    if len(xGuess) > len(bonds):
        xGuess = xGuess[1:]
#         xGuess = [0.05] * len(vcBonds)
    elif len(xGuess) < len(bonds):
        xGuess = [0.05] * len(bonds)
#         np.concatenate(xGuess, [0.1])
#     print(len(xGuess), len(bonds))
    
    fwdsOriginais = fsolve(PrecoBondMatrizMin,xGuess)
    xGuess = fwdsOriginais
#     print(fwdsOriginais)
#     print(dict(zip(dtVctoBond, fwdsOriginais)))
    outputFwdOriginais.append(dict(zip(dtVctoBond, fwdsOriginais)))
    
    # Calculo das fwds para os prazos fixos
    fwdFixos = CalculaFwdPzFixo(fwdsOriginais, dtVctoBond, dtToday, listDtVctoFixo)
#     print(fwdFixos)
    outputFwd.append(dict(zip(listDtVctoFixo, fwdFixos)))

#     if i_aux == 5:
#         break
    i_aux += 1
#     print(index, xGuess)
print("Taxas calculadas com sucesso!")


Taxas calculadas com sucesso!


### Criar um arquivo de output em csv com data no index e prazos fixos no nome da coluna

In [14]:
dfOutputFwd = pd.DataFrame(outputFwd, index=dfPrecos.index, columns=listDtVctoFixo)
dfOutputFwd.index = pd.to_datetime(dfOutputFwd.index)
dfOutputFwd.to_csv(INPUT_PATH + OUTPUT_FILE)
print("Taxas Fwd Fixas exportadas!")


dfOutputFwdOriginais = pd.DataFrame(outputFwdOriginais, index=dfPrecos.index, columns=vcBonds)
dfOutputFwdOriginais.index = pd.to_datetime(dfOutputFwdOriginais.index)
dfOutputFwdOriginais.to_csv(INPUT_PATH + OUTPUT_FILE_REAL)
print("Taxas Fwd Originais exportadas!")

Taxas Fwd Fixas exportadas!
Taxas Fwd Originais exportadas!


# Calculo para as LTNs

In [66]:
# Definindo algumas variáveis
INPUT_FILES = ['LTN_2013.csv', 'LTN_2014.csv', 'LTN_2015.csv', 'LTN_2016.csv', 'LTN_2017.csv', 'LTN_2018.csv']
INPUT_PATH = "D:\\Documents\\JupyterNB\\tese_fgv\\dados\\dados_tratados\\"
INPUT_FILE = INPUT_FILES[5]
VNA_FILE = "VNAHist.csv"
OUTPUT_FILE = "fwdNominal_" + INPUT_FILE.split(".")[0].split("_")[1] + ".csv"
OUTPUT_FILE_REAL = "fwdNominalOriginal_" + INPUT_FILE.split(".")[0].split("_")[1] + ".csv"
MONTHS_PGTO_COUPON = [1]
listDtVctoFixo = [360, 5*360, 10*360, 20*360]

# Importando dados
dfPrecos = pd.read_csv(INPUT_PATH + INPUT_FILE, index_col='Dia')

In [67]:
vcBonds = list(dfPrecos.columns)
xGuess = [0.015] * len(vcBonds)
outputFwd  = []
outputFwdOriginais = []
# TESTA SE É O ARQUIVO É DE NTNB (com fluxo de caixa) OU LTN (sem fluxo de caixa)
if ('LTN' in INPUT_FILE):
    freqCupom = 1
elif ('Principal' in INPUT_FILE):
    freqCupom = 1
else:
    freqCupom = 2

i_aux = 0 # utilizado para fazer pontos de parada no for

# Inicio do calculo das fwds para cada um dos dias
# for index, row in dfPrecos.loc['2013-05-05':'2013-05-06'].iterrows():
for index, row in dfPrecos.iterrows():
    bonds = []
    b =[]
#     print(index)
    
    # Inicia a data base e VNA
    dtToday = datetime.strptime(index, "%Y-%m-%d")
    VNA = AtualizaVNA(index)
    
    for vcBond in vcBonds:
        # Cria o dicionário com os preços e vencimentos para cada dia
        if datetime.strptime(vcBond, '%d/%m/%Y').strftime('%Y-%m-%d') >= index and not pd.isna((row[vcBond])): # verifica se tem algum bond vencido 
            bonds.append({'preco':row[vcBond], 'dtVcto':datetime.strptime(vcBond, '%d/%m/%Y').strftime('%Y-%m-%d'),
                           'freqCoupon': freqCupom})
            b.append(row[vcBond])    
#     print(bonds)
    # Capturo algumas informações para o calculo das matrizes
    dtVctoBond = []
    for bond in bonds:
        dtVctoBond.append(datetime.strptime(bond['dtVcto'], '%Y-%m-%d'))

    maxVcto = max(dtVctoBond)
    dayVcto, monthVcto, yearVcto = maxVcto.day, maxVcto.month, maxVcto.year
    dayBase, monthBase, yearBase = dtToday.day, dtToday.month, dtToday.year

    # Calculo da matriz de cupons e os prazos
    mtzCoupon, PzPgtoCupom = CalculaMatriz(dtToday, maxVcto, bonds, MONTHS_PGTO_COUPON, INPUT_FILE)
  
    # criando o vetor de cupons
    vlCupom = ((1 + 0.0)**(0.5) -1) * VNA
    arrayvlCupom = [vlCupom] * len(PzPgtoCupom)

    mtzPrincipal = CalculaMatrizPrincipal(VNA,PzPgtoCupom, bonds, INPUT_FILE)

    # Somando a matriz de cupons e de principais
    mtzVlCupom = (arrayvlCupom * mtzCoupon) + mtzPrincipal

    # Calculo das fwds para os vctos dos bonds (taxas a.a base 360)
#     print(xGuess)
    if len(xGuess) > len(bonds):
        xGuess = xGuess[0:len(bonds)]
    elif len(xGuess) <= len(bonds):
        xGuess = [0.05] * len(bonds)
    
#     print(xGuess)
    fwdsOriginais = fsolve(PrecoBondMatrizMin,xGuess)
    xGuess = fwdsOriginais
    outputFwdOriginais.append(dict(zip(dtVctoBond, fwdsOriginais)))
    
    # Calculo das fwds para os prazos fixos
    fwdFixos = CalculaFwdPzFixo(fwdsOriginais, dtVctoBond, dtToday, listDtVctoFixo)
#     print(dict(zip(listDtVctoFixo, fwdFixos)))
    outputFwd.append(dict(zip(listDtVctoFixo, fwdFixos)))

    i_aux += 1
    
print("Taxas calculadas com sucesso!")


Taxas calculadas com sucesso!


### Criar um arquivo de output em csv com data no index e prazos fixos no nome da coluna

In [68]:
dfOutputFwd = pd.DataFrame(outputFwd, index=dfPrecos.index, columns=listDtVctoFixo)
dfOutputFwd.index = pd.to_datetime(dfOutputFwd.index)
dfOutputFwd.to_csv(INPUT_PATH + OUTPUT_FILE)
print("Taxas Fwd Fixas exportadas!")


dfOutputFwdOriginais = pd.DataFrame(outputFwdOriginais, index=dfPrecos.index, columns=vcBonds)
dfOutputFwdOriginais.index = pd.to_datetime(dfOutputFwdOriginais.index)
dfOutputFwdOriginais.to_csv(INPUT_PATH + OUTPUT_FILE_REAL)
print("Taxas Fwd Originais exportadas!")

Taxas Fwd Fixas exportadas!
Taxas Fwd Originais exportadas!


In [ ]:
dfOutputFwd = pd.DataFrame(outputFwd, index=dfPrecos.index)
dfOutputFwdOriginais = pd.DataFrame(outputFwdOriginais, index=dfPrecos.index)

dfOutputFwdOriginais.index = pd.to_datetime(dfOutputFwdOriginais.index)
plt.plot(dfOutputFwdOriginais.index.tolist(), dfOutputFwdOriginais)
plt.show()
# # dfOutputFwd.index.tolist()
# dfOutputFwd
dfOutputFwdOriginais.describe()
dfOutputFwdOriginais

In [ ]:
# outputFwdOriginais
# dfOutputFwdOriginais[dfOutputFwdOriginais['15/05/2013'] == -0.235048]
# dfOutputFwdOriginais
# dfOutputFwd['2013-05-10':'2013-05-16']

In [ ]:
dfOutputFwd.index = pd.to_datetime(dfOutputFwd.index)
plt.plot(dfOutputFwd.index.tolist(), dfOutputFwd)
plt.show()
# # dfOutputFwd.index.tolist()
# dfOutputFwd
dfOutputFwd.describe()
dfOutputFwd

# PCA das diferenças das curvas

In [ ]:
dfDiff = dfOutputFwd.diff().dropna()
dfDiff.shape

In [ ]:
from sklearn.decomposition import PCA
sklearn_pca = PCA(n_components=3)
Y_sklearn = sklearn_pca.fit_transform(dfDiff)

In [ ]:
Y_sklearn.shape

In [ ]:
sklearn_pca.components_, sklearn_pca.explained_variance_ratio_, sklearn_pca.explained_variance_

In [ ]:
sklearn_pca.components_, sklearn_pca.explained_variance_ratio_
features = range(sklearn_pca.n_components_)
plt.bar(features, sklearn_pca.explained_variance_ratio_)

In [ ]:
sklearn_pca = PCA(n_components=1)
Y_sklearn = sklearn_pca.fit_transform(dfDiff)

In [ ]:
Y_sklearn.shape

In [ ]:
# components_ são os autovetores
# 
sklearn_pca.components_, sklearn_pca.explained_variance_ratio_

In [ ]:
dtVctoBond, fwdsOriginais

In [ ]:
y
x2
d=[]
d.append(dict(zip(y, x2)))
d.append(dict(zip(y, x1)))
dfd = pd.DataFrame(d)
dfd